In [9]:
import nfl_data_py as nfl
import pandas as pd
import os
import urllib.request
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox
from matplotlib.offsetbox import OffsetImage
from PIL import Image
import numpy as np
from io import BytesIO
import requests
# ML Libraries
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas
import plotly.express as px

# Graphing 
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.pyplot import figure

In [10]:
#Import Data
nfl_data = nfl.import_schedules([2023])
original_df = nfl_data.groupby('week')

In [11]:
# Select relevant columns for team vs. team DataFrame
team_vs_team_df = nfl_data[['week', 'away_team', 'away_score', 'home_team', 'home_score']]


In [12]:
# Get data from games
weekly_nfl = nfl.import_weekly_data([2023])
weekly_nfl.head()

Downcasting floats.


,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NYJ,2023,1,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,0.00,0.00
1,00-0024243,M.Lewis,Marcedes Lewis,TE,TE,https://static.www.nfl.com/image/private/f_aut...,CHI,2023,4,REG,...,0.0,0.483465,0,0.0,0.031250,-0.012397,0.038197,0.0,0.80,1.80
2,00-0024243,M.Lewis,Marcedes Lewis,TE,TE,https://static.www.nfl.com/image/private/f_aut...,CHI,2023,7,REG,...,1.0,1.437224,0,3.2,0.034483,0.094340,0.117762,0.0,1.60,2.60
3,00-0024243,M.Lewis,Marcedes Lewis,TE,TE,https://static.www.nfl.com/image/private/f_aut...,CHI,2023,11,REG,...,0.0,-0.547367,0,0.0,0.045455,0.036885,0.094001,0.0,0.00,0.00
4,00-0026498,M.Stafford,Matthew Stafford,QB,QB,https://static.www.nfl.com/image/private/f_aut...,LA,2023,1,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,14.46,14.46


In [13]:
#clean up data to show data columns for each game
columns = [ 'week','recent_team','opponent_team','passing_epa',
           'passing_yards', 'rushing_epa','rushing_yards','receiving_epa'
           ]


In [14]:
#create Df
games_df = weekly_nfl[columns]
games_df.head()

,week,recent_team,opponent_team,passing_epa,passing_yards,rushing_epa,rushing_yards,receiving_epa
0,1,NYJ,BUF,-2.031960,0.0,NaN,0.0,NaN
1,4,CHI,DEN,NaN,0.0,NaN,0.0,0.483465
2,7,CHI,LV,NaN,0.0,NaN,0.0,1.437224
3,11,CHI,DET,NaN,0.0,NaN,0.0,-0.547367
4,1,LA,SEA,20.679981,334.0,0.868086,11.0,NaN


In [15]:
# Groupby team and week offense stats
team_by_week = games_df.groupby(['week', 'recent_team'])
team_by_week.head()
team_stats_per_game = team_by_week.sum()
team_stats_per_game.index.set_names(['week', 'team_offense'], inplace=True)
team_stats_per_game = team_stats_per_game.reset_index()
#team_stats_per_game_off = team_stats_per_game.drop(columns='index')

team_stats_per_game_off = team_stats_per_game.rename(columns={'team_defense': 'team_offense'})
# List of columns to add '_def' to
columns_to_suffix = ['passing_epa',
           'passing_yards', 'rushing_epa','rushing_yards','receiving_epa']

team_stats_per_game_off = team_stats_per_game_off.rename(columns={col: col + '_offense' for col in columns_to_suffix})

team_stats_per_game_off.tail()

,week,team_offense,passing_epa_offense,passing_yards_offense,rushing_epa_offense,rushing_yards_offense,receiving_epa_offense
321,11,SEA,-0.065491,236.0,-0.166391,68.0,4.650634
322,11,SF,15.870390,333.0,-3.784996,115.0,21.576029
323,11,TB,-9.188732,246.0,1.573847,66.0,1.552182
324,11,TEN,2.962959,158.0,-1.595661,92.0,8.118902
325,11,WAS,-8.883590,255.0,1.397827,174.0,-8.998522


In [16]:
# Groupby team and week defense stats
team_by_week_def = games_df.groupby(['week', 'opponent_team'])
team_by_week_def.head()
team_stats_per_game_def = team_by_week_def.sum()

# List of columns to add '_def' to
columns_to_suffix_def = ['passing_epa',
           'passing_yards', 'rushing_epa','rushing_yards','receiving_epa']

# Add '_def' suffix to the specified columns
team_stats_per_game_def = team_stats_per_game_def.rename(columns={col: col + '_def' for col in columns_to_suffix_def})
team_stats_per_game_def.index.set_names(['week', 'team_defense'], inplace=True)
team_stats_per_game_def = team_stats_per_game_def.reset_index()


In [17]:
team_stats_per_game_def.head()

,week,team_defense,passing_epa_def,passing_yards_def,rushing_epa_def,rushing_yards_def,receiving_epa_def
0,1,ARI,-13.413063,202.0,-5.708867,92.0,1.409773
1,1,ATL,-10.528563,146.0,-8.897445,154.0,-3.103000
2,1,BAL,-11.156192,242.0,-8.125071,72.0,2.796677
3,1,BUF,-5.221493,140.0,2.803440,172.0,3.475710
4,1,CAR,-4.450450,115.0,-0.595772,130.0,0.718799


In [18]:
# merge both dfs so it shows offense and defense stats per game
team_total_df = team_stats_per_game.merge(team_stats_per_game_def, left_on='team_offense', right_on='team_defense')
# Filter rows where 'team_offense' is equal to 'team_defense'
team_total_df = team_total_df[team_total_df['team_offense'] == team_total_df['team_defense']]
#remove duplicates
team_total_df = team_total_df[(team_total_df['team_offense'] == team_total_df['team_defense']) & (team_total_df['week_x'] == team_total_df['week_y'])]
team_total_df.head(1)

,week_x,team_offense,passing_epa,passing_yards,rushing_epa,rushing_yards,receiving_epa,week_y,team_defense,passing_epa_def,passing_yards_def,rushing_epa_def,rushing_yards_def,receiving_epa_def
0,1,ARI,-9.239668,132.0,-11.179296,96.0,-2.79934,1,ARI,-13.413063,202.0,-5.708867,92.0,1.409773


In [19]:
# create a copy of team_total_df for home team and away team

# Make the first copy of the original DataFrame
home_team_df = team_total_df.copy()

# Make a second copy from the first copy
away_team_df = home_team_df.copy()

In [20]:
#make a home_team df
# Add '_home' to each column name
for column in home_team_df.columns:
    new_column_name_home = column + '_home'
    home_team_df.rename(columns={column: new_column_name_home}, inplace=True)
    
home_team_df.head()

,week_x_home,team_offense_home,passing_epa_home,passing_yards_home,rushing_epa_home,rushing_yards_home,receiving_epa_home,week_y_home,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home
0,1,ARI,-9.239668,132.0,-11.179296,96.0,-2.799340,1,ARI,-13.413063,202.0,-5.708867,92.0,1.409773
12,2,ARI,7.430885,228.0,2.366217,151.0,7.653436,2,ARI,8.180348,321.0,6.719742,127.0,11.507128
24,3,ARI,4.560631,189.0,7.001422,222.0,8.408302,3,ARI,-3.659615,249.0,8.314601,185.0,0.522374
36,4,ARI,3.898311,265.0,5.478536,105.0,6.421503,4,ARI,22.844091,283.0,2.675849,124.0,24.983665
48,5,ARI,-15.651534,166.0,-1.152939,142.0,-5.404606,5,ARI,9.498447,317.0,-8.893337,93.0,16.416182


In [21]:
#make a away_team df
# Add '_away' to each column name
for column in away_team_df.columns:
    new_column_name_away = column + '_away'
    away_team_df.rename(columns={column: new_column_name_away}, inplace=True)
away_team_df.head()

,week_x_away,team_offense_away,passing_epa_away,passing_yards_away,rushing_epa_away,rushing_yards_away,receiving_epa_away,week_y_away,team_defense_away,passing_epa_def_away,passing_yards_def_away,rushing_epa_def_away,rushing_yards_def_away,receiving_epa_def_away
0,1,ARI,-9.239668,132.0,-11.179296,96.0,-2.799340,1,ARI,-13.413063,202.0,-5.708867,92.0,1.409773
12,2,ARI,7.430885,228.0,2.366217,151.0,7.653436,2,ARI,8.180348,321.0,6.719742,127.0,11.507128
24,3,ARI,4.560631,189.0,7.001422,222.0,8.408302,3,ARI,-3.659615,249.0,8.314601,185.0,0.522374
36,4,ARI,3.898311,265.0,5.478536,105.0,6.421503,4,ARI,22.844091,283.0,2.675849,124.0,24.983665
48,5,ARI,-15.651534,166.0,-1.152939,142.0,-5.404606,5,ARI,9.498447,317.0,-8.893337,93.0,16.416182


In [22]:
# Make df with points and stats

# merge home team with home stats
home_team = team_vs_team_df.merge(home_team_df, left_on='home_team', right_on='team_offense_home')
#remove duplicates
home_team = home_team[(home_team['home_team'] ==  home_team['team_offense_home']) & (home_team['week_x_home'] == home_team['week'])]
home_team.tail(10)

,week,away_team,away_score,home_team,home_score,week_x_home,team_offense_home,passing_epa_home,passing_yards_home,rushing_epa_home,rushing_yards_home,receiving_epa_home,week_y_home,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home
2583,3,DEN,20.0,MIA,70.0,3,MIA,31.062447,376.0,12.792768,350.0,31.062447,3,MIA,2.208719,306.0,-5.999615,69.0,-5.733638
2595,5,NYG,16.0,MIA,31.0,5,MIA,-1.844198,308.0,7.101428,222.0,-0.270984,5,MIA,-2.154711,205.0,-9.014591,85.0,9.488153
2606,6,CAR,21.0,MIA,42.0,6,MIA,7.147842,262.0,2.244307,162.0,9.160931,6,MIA,-12.907183,224.0,3.199303,108.0,-3.217727
2618,8,NE,17.0,MIA,31.0,8,MIA,12.581885,324.0,-11.498557,78.0,16.415697,8,MIA,-9.127332,161.0,-0.303921,77.0,-4.306061
2630,11,LV,13.0,MIA,20.0,11,MIA,4.769229,325.0,-7.673799,99.0,3.493786,11,MIA,-9.576633,271.0,-5.404755,36.0,-1.643759
2673,3,PIT,23.0,LV,18.0,3,LV,-8.352474,324.0,-2.516379,69.0,-2.013242,3,LV,5.921596,235.0,-6.737266,105.0,9.555780
2686,5,GB,13.0,LV,17.0,5,LV,-1.803250,208.0,-3.381476,96.0,6.604667,5,LV,-14.767113,182.0,4.087527,110.0,-11.481108
2698,6,NE,17.0,LV,21.0,6,LV,9.882565,264.0,-10.972410,84.0,11.559125,6,LV,-4.471819,200.0,4.164759,83.0,3.130720
2712,9,NYG,6.0,LV,30.0,9,LV,7.603049,209.0,-7.482466,125.0,9.152975,9,LV,-19.027122,200.0,-4.605570,123.0,-7.132212
2724,10,NYJ,12.0,LV,16.0,10,LV,-9.807395,153.0,-1.637336,148.0,-3.269739,10,LV,-4.556659,263.0,-3.730515,108.0,0.062029


In [23]:
# merge away team with home stats
away_team = home_team.merge(away_team_df, left_on='away_team', right_on='team_offense_away')
#remove duplicates
away_team = away_team[(away_team['away_team'] ==  away_team['team_offense_away']) & (away_team['week_x_away'] == away_team['week'])]
away_team.head(10)

,week,away_team,away_score,home_team,home_score,week_x_home,team_offense_home,passing_epa_home,passing_yards_home,rushing_epa_home,...,rushing_epa_away,rushing_yards_away,receiving_epa_away,week_y_away,team_defense_away,passing_epa_def_away,passing_yards_def_away,rushing_epa_def_away,rushing_yards_def_away,receiving_epa_def_away
0,1,DET,21.0,KC,20.0,1,KC,-6.453613,226.0,-2.581925,...,-3.636585,118.0,6.054341,1,DET,-6.453613,226.0,-2.581925,90.0,-5.052831
16,7,DET,6.0,BAL,38.0,7,BAL,24.334782,357.0,1.219639,...,1.834399,84.0,0.542692,7,DET,24.334782,357.0,1.219639,146.0,24.334782
28,10,DET,41.0,LAC,38.0,10,LAC,14.587980,323.0,-0.869600,...,11.664154,200.0,19.774326,10,DET,14.587980,323.0,-0.869600,98.0,14.587980
35,6,DET,20.0,TB,6.0,6,TB,-8.300112,206.0,-4.832714,...,-7.840065,40.0,22.679295,6,DET,-8.300112,206.0,-4.832714,46.0,-7.366295
43,4,DET,34.0,GB,20.0,4,GB,-12.693180,246.0,0.423209,...,4.333835,211.0,6.932714,4,DET,-12.693180,246.0,0.423209,27.0,-5.993364
52,3,CHI,10.0,KC,41.0,3,KC,14.211285,303.0,0.524553,...,2.051218,116.0,-9.638733,3,CHI,14.211285,303.0,0.524553,153.0,14.669461
69,9,CHI,17.0,NO,24.0,9,NO,3.519515,214.0,-12.226431,...,3.474334,156.0,-1.557883,9,CHI,3.519515,214.0,-12.226431,87.0,4.718380
76,5,CHI,40.0,WAS,20.0,5,WAS,3.675323,388.0,-5.300707,...,8.263792,178.0,17.594566,5,CHI,3.675323,388.0,-5.300707,29.0,5.126959
90,8,CHI,13.0,LAC,30.0,8,LAC,17.087563,298.0,-14.207475,...,0.061330,73.0,-6.471786,8,CHI,17.087563,298.0,-14.207475,54.0,13.820724
104,11,CHI,26.0,DET,31.0,11,DET,-2.145055,236.0,3.663200,...,0.652613,183.0,5.655354,11,CHI,-2.145055,236.0,3.663200,115.0,3.821503


In [24]:
# rename df
full_df = away_team.copy()

In [25]:
full_df.to_excel('training_nfl.xlsx', index=False)


In [26]:
# Create Columns for training
column_train_x = [
   'passing_epa_home','passing_yards_home','rushing_epa_home','rushing_yards_home','receiving_epa_home',
    'passing_epa_def_home','passing_yards_def_home','rushing_epa_def_home','rushing_yards_def_home','receiving_epa_def_home',
       'passing_epa_away','passing_yards_away','rushing_epa_away','rushing_yards_away','receiving_epa_away',
           'passing_epa_def_away','passing_yards_def_away','rushing_epa_def_away','rushing_yards_def_away','receiving_epa_def_away',


]




column_train_y = ['home_score','away_score']

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# model
# Split data into features (X) and target variable (y)
X = full_df[column_train_x]
y = full_df[column_train_y]

# Encode categorical variables using one-hot encoding if needed

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Create a Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2): {r2}")


Mean Absolute Error (MAE): 4.599579807483789
Mean Squared Error (MSE): 32.76930322234941
Root Mean Squared Error (RMSE): 5.723745249063976
R-squared (R2): 0.6387027846858123


In [28]:
team_stats_per_game_off.head()

,week,team_offense,passing_epa_offense,passing_yards_offense,rushing_epa_offense,rushing_yards_offense,receiving_epa_offense
0,1,ARI,-9.239668,132.0,-11.179296,96.0,-2.799340
1,1,ATL,-4.450450,115.0,-0.595772,130.0,0.718799
2,1,BAL,-4.415663,169.0,0.155152,110.0,7.985488
3,1,BUF,-1.409397,236.0,-3.168568,97.0,5.100840
4,1,CAR,-10.528563,146.0,-8.897445,154.0,-3.103000


In [29]:
### Create mean of these stats to create test data
#home team
mean_offense = team_stats_per_game_off.groupby('team_offense').mean()
mean_offense = mean_offense.drop('week', axis=1)
mean_offense = mean_offense.reset_index()

mean_offense.head()

,team_offense,passing_epa_offense,passing_yards_offense,rushing_epa_offense,rushing_yards_offense,receiving_epa_offense
0,ARI,-8.604815,190.363632,0.047463,126.181816,-2.066517
1,ATL,-3.266922,226.000000,-3.026095,130.399994,2.611170
2,BAL,2.174069,225.363632,1.267984,155.090912,6.894430
3,BUF,6.938099,261.363647,1.964198,117.727272,8.490095
4,CAR,-9.621664,205.100006,-1.716738,92.300003,0.325570


In [30]:
#defense team
# Split the DataFrame into non-numerical and numerical columns
mean_defense = team_stats_per_game_def.groupby('team_defense').mean()
# Remove the week column
mean_defense = mean_defense.drop('week', axis=1)
mean_defense = mean_defense.reset_index()

mean_defense.head()


,team_defense,passing_epa_def,passing_yards_def,rushing_epa_def,rushing_yards_def,receiving_epa_def
0,ARI,2.498649,238.636368,-0.276411,132.090912,8.027170
1,ATL,0.173111,214.500000,-3.599363,108.400002,5.315990
2,BAL,-8.237186,201.545456,-1.867474,103.727272,0.200522
3,BUF,-1.176736,224.909088,-2.137583,110.454544,4.375284
4,CAR,0.717090,191.699997,1.750396,129.399994,3.874348


In [31]:
# Merge on defense vs offense per team
test_data_df = pd.merge(mean_defense, mean_offense, left_on='team_defense',right_on='team_offense', how='left')
test_data_df.head()

,team_defense,passing_epa_def,passing_yards_def,rushing_epa_def,rushing_yards_def,receiving_epa_def,team_offense,passing_epa_offense,passing_yards_offense,rushing_epa_offense,rushing_yards_offense,receiving_epa_offense
0,ARI,2.498649,238.636368,-0.276411,132.090912,8.027170,ARI,-8.604815,190.363632,0.047463,126.181816,-2.066517
1,ATL,0.173111,214.500000,-3.599363,108.400002,5.315990,ATL,-3.266922,226.000000,-3.026095,130.399994,2.611170
2,BAL,-8.237186,201.545456,-1.867474,103.727272,0.200522,BAL,2.174069,225.363632,1.267984,155.090912,6.894430
3,BUF,-1.176736,224.909088,-2.137583,110.454544,4.375284,BUF,6.938099,261.363647,1.964198,117.727272,8.490095
4,CAR,0.717090,191.699997,1.750396,129.399994,3.874348,CAR,-9.621664,205.100006,-1.716738,92.300003,0.325570


In [43]:
# create DF for upcoming week
# create stats to show upcpoming games
grouped = nfl_data.groupby('week')

# Select the group corresponding to week 6
week_6_data = grouped.get_group(12)
get_teams =['home_team','away_team','home_score','away_score']
schedule_week_df = week_6_data[get_teams]
schedule_week_df.reset_index(drop=True, inplace=True)
schedule_week_df.head()

,home_team,away_team,home_score,away_score
0,DET,GB,NaN,NaN
1,DAL,WAS,NaN,NaN
2,SEA,SF,NaN,NaN
3,NYJ,MIA,NaN,NaN
4,ATL,NO,NaN,NaN


In [44]:
# create a copy of team_total_df for home team and away team for testing

# Make the first copy of the original DataFrame
home_team_test = test_data_df.copy()

# Make a second copy from the first copy
away_team_test = home_team_test.copy()

In [45]:
# Add '_home' to each column name
for column in home_team_test.columns:
    new_column_name_test = column + '_home'
    home_team_test.rename(columns={column: new_column_name_test}, inplace=True)
    
home_team_test.head()

,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home,team_offense_home,passing_epa_offense_home,passing_yards_offense_home,rushing_epa_offense_home,rushing_yards_offense_home,receiving_epa_offense_home
0,ARI,2.498649,238.636368,-0.276411,132.090912,8.027170,ARI,-8.604815,190.363632,0.047463,126.181816,-2.066517
1,ATL,0.173111,214.500000,-3.599363,108.400002,5.315990,ATL,-3.266922,226.000000,-3.026095,130.399994,2.611170
2,BAL,-8.237186,201.545456,-1.867474,103.727272,0.200522,BAL,2.174069,225.363632,1.267984,155.090912,6.894430
3,BUF,-1.176736,224.909088,-2.137583,110.454544,4.375284,BUF,6.938099,261.363647,1.964198,117.727272,8.490095
4,CAR,0.717090,191.699997,1.750396,129.399994,3.874348,CAR,-9.621664,205.100006,-1.716738,92.300003,0.325570


In [46]:
# Add '_away' to each column name
for column in away_team_test.columns:
    new_column_name_test_away = column + '_away'
    away_team_test.rename(columns={column: new_column_name_test_away}, inplace=True)
    
away_team_test.head()

,team_defense_away,passing_epa_def_away,passing_yards_def_away,rushing_epa_def_away,rushing_yards_def_away,receiving_epa_def_away,team_offense_away,passing_epa_offense_away,passing_yards_offense_away,rushing_epa_offense_away,rushing_yards_offense_away,receiving_epa_offense_away
0,ARI,2.498649,238.636368,-0.276411,132.090912,8.027170,ARI,-8.604815,190.363632,0.047463,126.181816,-2.066517
1,ATL,0.173111,214.500000,-3.599363,108.400002,5.315990,ATL,-3.266922,226.000000,-3.026095,130.399994,2.611170
2,BAL,-8.237186,201.545456,-1.867474,103.727272,0.200522,BAL,2.174069,225.363632,1.267984,155.090912,6.894430
3,BUF,-1.176736,224.909088,-2.137583,110.454544,4.375284,BUF,6.938099,261.363647,1.964198,117.727272,8.490095
4,CAR,0.717090,191.699997,1.750396,129.399994,3.874348,CAR,-9.621664,205.100006,-1.716738,92.300003,0.325570


In [47]:
# Make df with points and stats for testing upcoming week

# merge home team with home stats
testing_home = schedule_week_df.merge(home_team_test, left_on='home_team', right_on='team_offense_home')
#remove duplicates
#testing_home = home_team[(home_team['home_team'] ==  home_team['team_offense_home']) & (home_team['week_x_home'] == home_team['week'])]
testing_home.head(1)

,home_team,away_team,home_score,away_score,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home,team_offense_home,passing_epa_offense_home,passing_yards_offense_home,rushing_epa_offense_home,rushing_yards_offense_home,receiving_epa_offense_home
0,DET,GB,NaN,NaN,DET,-0.831884,242.899994,-1.61521,89.5,3.950236,DET,4.430013,274.299988,-0.711561,136.600006,8.258707


In [48]:
# merge away team with home stats
testing_away = testing_home.merge(away_team_test, left_on='away_team', right_on='team_offense_away')
#remove duplicates
#away_team = away_team[(away_team['away_team'] ==  away_team['team_offense_away']) & (away_team['week_x_away'] == away_team['week'])]
testing_away.head(10)

,home_team,away_team,home_score,away_score,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home,...,passing_yards_def_away,rushing_epa_def_away,rushing_yards_def_away,receiving_epa_def_away,team_offense_away,passing_epa_offense_away,passing_yards_offense_away,rushing_epa_offense_away,rushing_yards_offense_away,receiving_epa_offense_away
0,DET,GB,NaN,NaN,DET,-0.831884,242.899994,-1.615210,89.500000,3.950236,...,208.199997,-0.505544,134.699997,5.095420,GB,1.457706,234.500000,-2.504519,102.099998,4.902522
1,DAL,WAS,NaN,NaN,DAL,-6.803716,177.600006,-2.250749,109.199997,-0.944755,...,272.636353,-1.544374,114.272728,10.433714,WAS,-2.246763,276.181824,-0.415294,95.727272,4.882170
2,SEA,SF,NaN,NaN,SEA,-1.072625,254.000000,-1.349404,112.599998,5.371908,...,239.100006,-0.104241,81.400002,-1.133610,SF,9.189562,266.899994,-0.297240,132.699997,13.963272
3,NYJ,MIA,NaN,NaN,NYJ,-2.389406,194.199997,-3.335766,137.600006,3.083333,...,241.100006,-0.257961,99.199997,4.169714,MIA,8.390397,300.100006,-0.448370,143.000000,10.513066
4,ATL,NO,NaN,NaN,ATL,0.173111,214.500000,-3.599363,108.400002,5.315990,...,210.500000,-1.399781,113.599998,0.007028,NO,-1.161276,252.600006,-2.805077,100.000000,4.078490
5,CIN,PIT,NaN,NaN,CIN,-0.175720,263.700012,0.838352,138.300003,5.568332,...,256.600006,-0.814419,127.699997,2.961278,PIT,-6.030575,187.800003,-0.703223,110.099998,-0.999555
6,HOU,JAX,NaN,NaN,HOU,2.198321,258.500000,-2.312942,96.500000,6.133255,...,265.899994,-1.462568,87.000000,1.971136,JAX,0.070627,240.800003,-4.286762,108.699997,4.431062
7,IND,TB,NaN,NaN,IND,-1.457964,249.600006,-0.643089,129.699997,4.745677,...,290.100006,-3.584208,89.000000,9.053980,TB,2.331860,238.899994,-4.303776,76.900002,5.643270
8,NYG,NE,NaN,NaN,NYG,0.302245,250.545456,0.154327,135.090912,3.593404,...,241.199997,-4.506466,97.699997,8.842393,NE,-5.651992,213.500000,-2.249362,94.900002,-2.857319
9,TEN,CAR,NaN,NaN,TEN,5.420552,248.600006,-3.350314,112.300003,10.605356,...,191.699997,1.750396,129.399994,3.874348,CAR,-9.621664,205.100006,-1.716738,92.300003,0.325570


In [49]:
# rename df
full_test = testing_away.copy()
#full_test = full_test.reset_index()

full_test = full_test.rename(columns={
    'passing_epa_offense_away': 'passing_epa_away',
    'passing_yards_offense_away': 'passing_yards_away',
    'receiving_epa_offense_away': 'receiving_epa_away',
    'rushing_epa_offense_away': 'rushing_epa_away',
    'rushing_yards_offense_away': 'rushing_yards_away',
     'passing_epa_offense_home': 'passing_epa_home',
    'passing_yards_offense_home': 'passing_yards_home',
    'receiving_epa_offense_home': 'receiving_epa_home',
    'rushing_epa_offense_home': 'rushing_epa_home',
    'rushing_yards_offense_home': 'rushing_yards_home',
})
full_test.dtypes

home_team                  object
away_team                  object
home_score                float64
away_score                float64
team_defense_home          object
passing_epa_def_home      float32
passing_yards_def_home    float32
rushing_epa_def_home      float32
rushing_yards_def_home    float32
receiving_epa_def_home    float32
team_offense_home          object
passing_epa_home          float32
passing_yards_home        float32
rushing_epa_home          float32
rushing_yards_home        float32
receiving_epa_home        float32
team_defense_away          object
passing_epa_def_away      float32
passing_yards_def_away    float32
rushing_epa_def_away      float32
rushing_yards_def_away    float32
receiving_epa_def_away    float32
team_offense_away          object
passing_epa_away          float32
passing_yards_away        float32
rushing_epa_away          float32
rushing_yards_away        float32
receiving_epa_away        float32
dtype: object

In [50]:
column_test_x = ['passing_epa_def_away', 'passing_yards_def_away',
       'rushing_epa_def_away', 'rushing_yards_def_away',
       'receiving_epa_def_away','passing_epa_away', 'passing_yards_away',
    'rushing_epa_away', 'rushing_yards_away','receiving_epa_away',
     'passing_epa_def_home', 'passing_yards_def_home',
       'rushing_epa_def_home', 'rushing_yards_def_home',
       'receiving_epa_def_home','passing_epa_home', 'passing_yards_home',
    'rushing_epa_home', 'rushing_yards_home',
       'receiving_epa_home',            
                ]

In [51]:
# Now, use the trained model to predict score for df_test
X_test = full_test[column_test_x]
y_pred = model.predict(X_test)

# The predictions are stored in y_pred
print("Predicted Scores:")
print(y_pred)

Predicted Scores:
[[24.88445  19.77185 ]
 [26.515932 17.325941]
 [19.300797 23.29541 ]
 [18.914309 24.124102]
 [20.58818  20.459833]
 [22.23172  20.070316]
 [23.495787 19.758522]
 [22.69984  20.241879]
 [20.172815 19.091362]
 [21.719015 17.409573]
 [20.036108 22.283459]
 [19.632051 21.347193]
 [18.254238 21.065186]
 [22.619234 21.75609 ]
 [20.459742 24.195372]
 [23.464144 18.454918]]


/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [52]:
# add predicted scores into df: 
predict_cols = ['home_team','away_team']
df_predictions = full_test[predict_cols]
scores_df = pd.DataFrame(y_pred, columns=['home_score', 'away_score'])
df_predictions = pd.concat([df_predictions, scores_df], axis=1)
df_predictions['Over/Under'] = df_predictions['home_score'] + df_predictions['away_score']
# Assuming 'home_score', 'away_score', 'home_team', and 'away_team' are columns in your DataFrame
df_predictions['Favorite'] = np.where(df_predictions['home_score'] > df_predictions['away_score'], df_predictions['home_team'], df_predictions['away_team'])
df_predictions['Spread'] = df_predictions['away_score'] - df_predictions['home_score']
df_predictions['spread'] = df_predictions['Spread'].apply(lambda x: '-' + str(x) if x >= 0 else str(x))
df_predictions = df_predictions.drop('Spread', axis=1)

df_predictions.head(35)

,home_team,away_team,home_score,away_score,Over/Under,Favorite,spread
0,DET,GB,24.884451,19.771851,44.656303,DET,-5.112600326538086
1,DAL,WAS,26.515932,17.325941,43.841873,DAL,-9.189990997314453
2,SEA,SF,19.300797,23.295410,42.596207,SF,-3.9946136474609375
3,NYJ,MIA,18.914309,24.124102,43.038410,MIA,-5.2097930908203125
4,ATL,NO,20.588181,20.459833,41.048012,ATL,-0.12834739685058594
5,CIN,PIT,22.231720,20.070316,42.302036,CIN,-2.1614036560058594
6,HOU,JAX,23.495787,19.758522,43.254311,HOU,-3.737264633178711
7,IND,TB,22.699841,20.241879,42.941719,IND,-2.4579620361328125
8,NYG,NE,20.172815,19.091362,39.264175,NYG,-1.0814533233642578
9,TEN,CAR,21.719015,17.409573,39.128586,TEN,-4.309442520141602


In [53]:
df_predictions.to_excel('week_9_pred.xlsx', index=False)


In [ ]:
from flask import Flask, render_template
app = Flask(__name__)

@app.route('/')
def display_dataframe():
    return df_predictions.to_html()

if __name__ == '__main__':
    app.run()








